## IMPORTING LIBRARIES

In [69]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from fbprophet import Prophet

In [70]:
df= pd.read_csv('USDGBPEUR_vs_EGP.csv')

In [71]:
df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Type
0,0,2016-02-22,7.6808,7.8347,7.6797,7.6818,7.6818,USD
1,1,2016-02-23,7.8074,7.8157,7.7996,7.8064,7.8064,USD
2,2,2016-02-24,7.8051,7.8173,7.7992,7.8037,7.8037,USD
3,3,2016-02-25,7.8095,7.8161,7.7982,7.8099,7.8099,USD
4,4,2016-02-26,7.8032,7.8171,7.7990,7.8042,7.8042,USD


In [72]:
df= df.iloc[:,1:8]

In [73]:
df['Type'].value_counts()

GBP    1567
EUR    1567
USD    1567
Name: Type, dtype: int64

In [74]:
fig = px.line(df, x="Date", y="Close", color='Type')
fig.show()

The GBP curve has always been above than EUR and the EUR cove has always been above than USD except for a period
We can see that in November 2016 there was a sizeable rise in the exchange rates of USD, GBP and EUR against EGP. This is because on Novemeber 3, 2016 Central Bank of Egypt  devalued its currency by 48% and announced that it will be allowed to float – measures that meet a key demand by the International Monetary Fund in order to secure a $12bn (£9.6bn) loan over three years to overhaul its ailing economy.
Read more here: https://www.theguardian.com/world/2016/nov/03/egypt-devalues-currency-meet-imf-demands-loan

In March 2018, exchange rates of USD, GBP and EUR against EGP reached their lowest value in around three and a half years demonstrating that the economy was gathering momentum following a series of IMF-induced reforms that began in November 2016 with the floating of the pound. Read more here: https://www.focus-economics.com/countries/egypt/news/gdp/economy-continues-gathering-pace-in-the-january-march-period

As of Feb 2022 our data indicates the exchange rates of EUR/EGP and GBP/EGP have been in a downtrend for the past 1 year whereas in the case USD/EGP is 15.719 the rate has been constant for around a year.

## **USD vs EGP**

In [75]:
dollars= df[df['Type']=='USD']
dollars.head()

,Date,Open,High,Low,Close,Adj Close,Type
0,2016-02-22,7.6808,7.8347,7.6797,7.6818,7.6818,USD
1,2016-02-23,7.8074,7.8157,7.7996,7.8064,7.8064,USD
2,2016-02-24,7.8051,7.8173,7.7992,7.8037,7.8037,USD
3,2016-02-25,7.8095,7.8161,7.7982,7.8099,7.8099,USD
4,2016-02-26,7.8032,7.8171,7.7990,7.8042,7.8042,USD


In [76]:
dollars.shape

(1567, 7)

In [77]:
dollars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 0 to 1566
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1567 non-null   object 
 1   Open       1564 non-null   float64
 2   High       1564 non-null   float64
 3   Low        1564 non-null   float64
 4   Close      1564 non-null   float64
 5   Adj Close  1564 non-null   float64
 6   Type       1567 non-null   object 
dtypes: float64(5), object(2)
memory usage: 97.9+ KB


In [78]:
dollars.isnull().sum()

Date         0
Open         3
High         3
Low          3
Close        3
Adj Close    3
Type         0
dtype: int64

In [79]:
## DATA CLEANING
dollars= dollars.dropna()
dollars.shape

(1564, 7)

In [80]:
fig = px.line(dollars, x='Date', y="Close")
fig.show()

In [81]:
fig = px.histogram(dollars, x="Close")
fig.show()

In the past 6 years the exchange rate of USD against EGP has mostly been in the region of 15.5 to 18.5.
The missing values from 9.5 to 15 is due to the devaluation of Egyptian currency in November 2016 as mentioned before.

In [82]:
### Currency Forecasting using PROPHET

Prophet is an open-source package (for both Python and R) for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well . Prophet is open source software released by Facebook’s core Data Science team.
References: 
- https://towardsdatascience.com/apple-stock-and-bitcoin-price-predictions-using-fbs-prophet-for-beginners-python-96d5ec404b77

In [83]:
#Covnvert the dataframe into the prophet expected format.

dollars.rename(columns={'Close': 'y', 'Date': 'ds'},inplace=True)
dollars['y']=dollars['y'].astype(float)
df_prophet = Prophet(daily_seasonality=True)
df_prophet.fit(dollars)
future1=df_prophet.make_future_dataframe(periods=730,freq='D')
dollars_forecast=df_prophet.predict(future1)
dollars_forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-02-22,7.664880,7.349139,8.660479,7.664880,7.664880,0.346982,0.346982,0.346982,0.826046,...,0.025880,0.025880,0.025880,-0.504944,-0.504944,-0.504944,0.0,0.0,0.0,8.011862
1,2016-02-23,7.666760,7.338524,8.697338,7.666760,7.666760,0.353056,0.353056,0.353056,0.826046,...,0.046378,0.046378,0.046378,-0.519369,-0.519369,-0.519369,0.0,0.0,0.0,8.019815
2,2016-02-24,7.668639,7.309478,8.697984,7.668639,7.668639,0.343578,0.343578,0.343578,0.826046,...,0.046923,0.046923,0.046923,-0.529391,-0.529391,-0.529391,0.0,0.0,0.0,8.012217
3,2016-02-25,7.670518,7.320848,8.673958,7.670518,7.670518,0.332491,0.332491,0.332491,0.826046,...,0.041435,0.041435,0.041435,-0.534990,-0.534990,-0.534990,0.0,0.0,0.0,8.003010
4,2016-02-26,7.672397,7.330815,8.706749,7.672397,7.672397,0.335727,0.335727,0.335727,0.826046,...,0.045895,0.045895,0.045895,-0.536215,-0.536215,-0.536215,0.0,0.0,0.0,8.008124


In [84]:
fig2 = px.line(dollars_forecast, x="ds", y="yhat", title='Forecast of exchange rate of Dollars vs Egyptian pound')
fig2.show()

In [85]:
## Predict currency vs EGP on a certain date using function. Enter date in YYYY-MM-DD format

def predict_value(df, date):
    print(df[df['ds'] == pd.to_datetime(date)]['yhat'].values[0])

In [86]:
predict_value(dollars_forecast,'2022-02-22')

15.094925244301209


## **GBP vs EGP**

In [87]:
pounds= df[df['Type']=='GBP']
pounds.head()

,Date,Open,High,Low,Close,Adj Close,Type
1567,2016-02-22,10.955,11.164,10.955,10.956,10.956,GBP
1568,2016-02-23,11.046,11.051,10.984,11.044,11.044,GBP
1569,2016-02-24,10.933,10.946,10.834,10.935,10.935,GBP
1570,2016-02-25,10.875,10.929,10.851,10.878,10.878,GBP
1571,2016-02-26,10.898,10.954,10.823,10.898,10.898,GBP


In [88]:
pounds.shape

(1567, 7)

In [89]:
pounds.isnull().sum()

Date         0
Open         2
High         2
Low          2
Close        2
Adj Close    2
Type         0
dtype: int64

In [90]:
## DATA CLEANING
pounds= pounds.dropna()
pounds.shape

(1565, 7)

In [91]:
fig = px.line(pounds, x='Date', y="Close")
fig.show()

In [92]:
fig = px.histogram(pounds, x="Close")
fig.show()

In the past 6 years the exchange rate of GBP against EGP has mostly been in the region of 19 to 25.
The missing values from 13.5 to 17.5 is due to the devaluation of Egyptian currency in November 2016 as mentioned before.

In [93]:
pounds.rename(columns={'Close': 'y', 'Date': 'ds'},inplace=True)
pounds['y']=pounds['y'].astype(float)
df_prophet = Prophet(daily_seasonality=True)
df_prophet.fit(pounds)
future2=df_prophet.make_future_dataframe(periods=730,freq='D')
pounds_forecast=df_prophet.predict(future2)
pounds_forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-02-22,10.910885,10.410544,12.686341,10.910885,10.910885,0.649962,0.649962,0.649962,0.973622,...,0.017450,0.017450,0.017450,-0.341110,-0.341110,-0.341110,0.0,0.0,0.0,11.560847
1,2016-02-23,10.912882,10.544171,12.715590,10.912882,10.912882,0.667358,0.667358,0.667358,0.973622,...,0.050080,0.050080,0.050080,-0.356345,-0.356345,-0.356345,0.0,0.0,0.0,11.580240
2,2016-02-24,10.914879,10.542999,12.635487,10.914879,10.914879,0.665561,0.665561,0.665561,0.973622,...,0.058261,0.058261,0.058261,-0.366322,-0.366322,-0.366322,0.0,0.0,0.0,11.580440
3,2016-02-25,10.916876,10.503759,12.794450,10.916876,10.916876,0.660560,0.660560,0.660560,0.973622,...,0.058084,0.058084,0.058084,-0.371146,-0.371146,-0.371146,0.0,0.0,0.0,11.577436
4,2016-02-26,10.918874,10.470095,12.770209,10.918874,10.918874,0.662148,0.662148,0.662148,0.973622,...,0.059530,0.059530,0.059530,-0.371004,-0.371004,-0.371004,0.0,0.0,0.0,11.581022


In [94]:
predict_value(pounds_forecast,'2022-02-22')

21.136609109889314


In [95]:
fig2 = px.line(pounds_forecast,  x="ds", y="yhat", title='Forecast of exchange rate of Pound Sterling vs Egyptian Pound')
fig2.show()

## **EUROS vs EGP**

In [96]:
euros= df[df['Type']=='EUR']
euros.shape

(1567, 7)

In [97]:
euros.isnull().sum()

Date         0
Open         3
High         3
Low          3
Close        3
Adj Close    3
Type         0
dtype: int64

In [98]:
## DATA CLEANING
euros= euros.dropna()
euros.shape

(1564, 7)

In [99]:
fig = px.line(euros, x='Date', y="Close")
fig.show()

In [100]:
fig = px.histogram(euros, x="Close")
fig.show()

In the past 6 years the exchange rate of GBP against EGP has mostly been in the region of 16.5 to 22. 
The missing values from 10.5 to 16.5 is due to the devaluation of Egyptian currency in November 2016 as mentioned before.

In [101]:
euros.rename(columns={'Close': 'y', 'Date': 'ds'},inplace=True)
euros['y']=euros['y'].astype(float)
df_prophet = Prophet(daily_seasonality=True)
df_prophet.fit(euros)
future2=df_prophet.make_future_dataframe(periods=730,freq='D')
euros_forecast=df_prophet.predict(future2)
euros_forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-02-22,8.513814,8.244137,9.823849,8.513814,8.513814,0.490911,0.490911,0.490911,0.918372,...,0.022585,0.022585,0.022585,-0.450046,-0.450046,-0.450046,0.0,0.0,0.0,9.004725
1,2016-02-23,8.516632,8.212685,9.793665,8.516632,8.516632,0.506190,0.506190,0.506190,0.918372,...,0.055061,0.055061,0.055061,-0.467242,-0.467242,-0.467242,0.0,0.0,0.0,9.022823
2,2016-02-24,8.519451,8.314888,9.775168,8.519451,8.519451,0.494503,0.494503,0.494503,0.918372,...,0.055045,0.055045,0.055045,-0.478914,-0.478914,-0.478914,0.0,0.0,0.0,9.013953
3,2016-02-25,8.522269,8.259638,9.738082,8.522269,8.522269,0.480984,0.480984,0.480984,0.918372,...,0.047663,0.047663,0.047663,-0.485050,-0.485050,-0.485050,0.0,0.0,0.0,9.003253
4,2016-02-26,8.525087,8.219493,9.801377,8.525087,8.525087,0.481882,0.481882,0.481882,0.918372,...,0.049240,0.049240,0.049240,-0.485730,-0.485730,-0.485730,0.0,0.0,0.0,9.006969


In [102]:
predict_value(euros_forecast,'2022-02-22')

17.342441290904464


In [103]:
fig2 = px.line(euros_forecast,  x="ds", y="yhat", title='Forecast of exchange rate of Euros vs Egyptian Pound')
fig2.show()